# **Python CAD**

### **Packages**

In [217]:
import ezdxf
import pandas as pd
from ezdxf import zoom

### **Import data**

In [218]:
file_name='NV-22.MD'
df=pd.read_csv(f'{file_name}.CSV', usecols=[0,1,2,3,4], names=["est", "x", "y", "z", "des"], header=None)
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)

df['des']=df['des'].str.upper()
df['xy']=df.apply(lambda j: (j['x'],j['y']),axis=1)
df['xyz']=df.apply(lambda j: (j['x'],j['y'],j['z']),axis=1)
#df.head()

### **Config DXF file**

In [219]:
# Argument setup=True for adding standard linetypes and text styles.
doc=ezdxf.new(dxfversion='R2000', setup=True)
#doc=ezdxf.new(dxfversion='R12', setup=True)
doc.header['$PDMODE'] = 35
doc.header['$PDSIZE'] = 0.5

#Layes dxfattribs
layers_dxfattribs = {
    'L' :{'color':1},
    'T' :{'color':6},
    'P' :{'color':2},
    'L1':{'color':4},
    'L2':{'color':3},
    'L3':{'color':5},
    'L4':{'color':40},
    'SEC':{'color':7},
    'PP':{'color':188},
    '2D':{'color':234, 'linetype':'DASHED','lineweight':0.1}

}

msp=doc.modelspace()

### **Sections**

In [220]:
#Select only sections
filt = (df['des']=='SEC')
df_sec= df[filt][['est','xyz','des']]

#find start : end of sections
est_list = df_sec['est'].tolist()
est_list = [int(x) for x in est_list]

idx=[]
for i,v in enumerate(est_list):
    if v==1:
        idx=idx+[i]

print(idx)

#Build layer 'SEC'
if len(idx)>0:
    doc.layers.new(name='SEC', dxfattribs=layers_dxfattribs['SEC'])

#Draw sections
for i in range(len(idx)):
    if i < len(idx)-1:
        points = df_sec['xyz'][idx[i]:idx[i+1]].tolist()
        msp.add_polyline3d(points=points, close=True, dxfattribs={'layer':'SEC'})
        
    else:
        points = df_sec['xyz'][idx[i]:].tolist()
        #pprint(points)
        msp.add_polyline3d(points=points, close=True, dxfattribs={'layer':'SEC'})

[0, 13, 30, 48, 65, 80, 107, 122, 138]


### **LTP**

In [221]:
#Select only LTP
cases = ['L','L1','L2','L3','L4','T','P']
filt = df['des'].isin(cases)
df_ltp = df.loc[filt,['des','xyz','xy']]
df_ltp.reset_index(drop=True, inplace=True)

#find start : end of LTP
ltp_list = df_ltp['des'].tolist()

idx=[]
jo = 7777
vo = 'any'

for i,v in enumerate(ltp_list):
    if v!=vo:
        idx=idx+[i]
        jo=i
        vo=v

#Build layers
for l in df_ltp['des'].unique():
    doc.layers.new(name=l, dxfattribs=layers_dxfattribs[l])
    if l=='L':
        doc.layers.new(name='2D', dxfattribs=layers_dxfattribs['2D'])

#Draw LTP
for i in range(len(idx)):
    if i < len(idx)-1:
        points = df_ltp['xyz'][idx[i]:idx[i+1]].tolist()
        msp.add_polyline3d(points=points, close=False, dxfattribs={'layer':df_ltp['des'][idx[i]]})
        if df_ltp['des'][idx[i]]=='L':
            points_2d = df_ltp['xy'][idx[i]:idx[i+1]].tolist()
            msp.add_lwpolyline(points=points_2d,dxfattribs={'layer':'2D'})
        
    else:
        points = df_ltp['xyz'][idx[i]:].tolist()
        msp.add_polyline3d(points=points, close=False, dxfattribs={'layer':df_ltp['des'][idx[i]]})
        if df_ltp['des'][idx[i]]=='L':
            points_2d = df_ltp['xy'][idx[i]:].tolist()
            msp.add_lwpolyline(points=points_2d,dxfattribs={'layer':'2D'})

### **3D Points**

In [222]:
#Select only points
filt = (df['des']=='PP')
df_pp= df[filt][['est','xyz','xy']]
pps = df_pp.values.tolist()

#Build layer 'PP'
if len(pps)>0:
    doc.layers.new(name='PP', dxfattribs=layers_dxfattribs['PP'])

#Draw 3D points
for pp in pps:
    msp.add_point(pp[1], dxfattribs={'layer':'PP'})
    msp.add_text(pp[0],
                dxfattribs={
                    'layer':'PP',
                    'style': 'LiberationSerif',
                    'height': 0.5}
                ).set_pos(pp[1], align='LEFT')

#Draw 2D Points
if 'PP' in doc.layers:
    for pp in pps:
        msp.add_point(pp[2], dxfattribs={'layer':'2D'})
        msp.add_text(pp[0],
                    dxfattribs={
                        'layer':'2D',
                        'style': 'LiberationSerif',
                        'height': 0.5}
                    ).set_pos(pp[2], align='LEFT')

### **Save as dxf file**

In [223]:
zoom.extents(msp)
doc.saveas('demo.dxf')

In [227]:
import os
files = os.listdir()
files_csv = []

for file in files:
    if file.endswith('.CSV'):
        files_csv.append(file)

files_csv
files_name = [f[:-4] for f in files_csv]
files_name

['NV-19A.MD', '20A.MD', '8A_1.MD', 'NV-19A.MD-MOD', 'NV-22.MD', 'NV-20A.MD']